In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
module_path = os.path.abspath(os.path.join('../../src/utils'))
if module_path not in sys.path:
    sys.path.append(module_path)
from simulator import Simulator

In [2]:
pd.options.display.float_format = '{:20,.5f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Part 1. data preparation

## 1). configurations: update the parameters at the below cell

In [3]:
# the modeling_stack file path used for the modelling
# input_file_path = '../../data/processed_input_data/02_modelling_stack_data_seasonality.csv'
input_file_path = '../../data/processed_input_data/02_modelling_stack_data_seasonality.csv'
# the 'model_outputs.xlsx' file path from the model_results
final_model_path = '../../models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/model_results_0802/model_outputs.xlsx'
# the 'variables_transformation_summary.xlsx' file path from the model_results
trans_path = '../../models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/model_results_0802/variables_transformation_summary.xlsx'

# Select the transformation method, c_curve or s_curve
# transformation_method = 'c_curve'
transformation_method = 's_curve'

# the total budget for each variable during the simulation period
# 1. the keys are the variables need to be simulated (e.g. 'M_SBA_SPEND') 
# 2. the first value is the total budget in terms of dollars for each driver during the simulation period (e.g. $1700000 for M_ON_DIS_AT_SPEND)
# 3. the second value is the variable metric which needs to be simulated (e.g. CLK)
# 4. the third value is the start date of the simulation for the variable (e.g. '2022-01-01')

media_spends_dict = {'M_ON_DIS_AT_SPEND' : [1700000, 'IMP', '2022-01-01'],
                    'M_ON_DIS_CT_ROS_SPEND' : [960000, 'IMP', '2022-01-01'],
                     'M_ON_DIS_HPLO_SPEND': [25000, 'IMP', '2022-01-01'],
                     'M_ON_DIS_KW_SPEND' : [117000, 'IMP', '2022-01-01'],
                     'M_OFF_DIS_WN_SPEND' : [1100000, 'IMP', '2022-01-01'],
                     'M_OFF_DIS_PIN_SPEND' : [95000, 'IMP', '2022-01-01'],
                     'M_NATIONAL_TV_SPEND': [900000, 'IMP', '2022-01-01'],
                     'M_SP_AB_SPEND' : [1375000, 'CLK', '2022-01-01'],
                     'M_SBA_SPEND' : [940000, 'CLK', '2022-01-01'],
                     'M_SP_KWB_SPEND':  [3200000, 'CLK', '2022-01-01']}

# media_spends_dict = {}
# About the HPLO variables
""" 
1). the rate for all the CPD variables = total_spend / total_on_air_days
2). the daily metrics for the home_page variable is: historical total impression/historical on-air days
3). below are the CPD variables:
    a). M_ON_DIS_HPLO_VIEW_IMP
    b). M_ON_DIS_HPLO_IMP
    c). M_ON_DIS_HPLO_CLK
    d). M_ON_DIS_HPLO_SPEND_CALC
    e). M_INSTORE_TV_WALL_IMP
    f). M_INSTORE_SCO_DETECTED_IMP
    g). M_INSTORE_SCO_IMP
    h). M_INSTORE_SCO_SPEND_CALC
"""

# the cost_average for the variable
# it will calculate the rates (cost average) from historical data by default
# users can update the rates for any media drivers at the below dict, 
# by adding the rate and the corresponding rate_unit, e.g. 'CPI', 'CPM', 'CPD', 'CPC'. 
media_cpm_cpi_dict = {'M_ON_DIS_ROS_SPEND': [0.005629447, 'CPI']}

# use the modeling metric here (IMP, CLK, etc.); do not use something else. 
# the key is the aggregrated variable in the modeling (e.g. M_SEARCH_MAN_CLK)
# the values are the the singular components for each aggregrated variable
granular_media_dict = {
                         'M_SEARCH_AUTO_CLK': ['M_SP_AB_CLK'],
                         'M_SEARCH_MAN_CLK': ['M_SBA_CLK', 'M_SP_KWB_CLK'],
                         }

# granular_media_dict = {}
# the simulation start and end dates, the simulation period should be less than 365 days
simulation_start_end_date = ('2022-01-01', '2022-12-31')

# the transformation start and end dates
modeling_start_end_date = ('2019-01-01', '2020-12-31')

# use adstock transformation or not
apply_adstock = False

# pre-process the data stack
df = pd.read_csv(input_file_path, thousands= ",", na_values = " -   ")
df['index'] = pd.to_datetime(df['index']) 
df.set_index("index", inplace = True, drop= False)
df['INTERCEPT_TERM'] = 1
# Please specify your custom variables that were not part of standard data stack
# Example:
df['M_SEARCH_MAN_CLK']=df['M_SBA_CLK']+df['M_SP_KWB_CLK']
df['M_SEARCH_AUTO_CLK']=df['M_SP_AB_CLK']
df['M_NATIONAL_TV_SPEND']=1000 
df['M_SEARCH_MAN_SPEND']=df['M_SBA_SPEND']+df['M_SP_KWB_SPEND']
df['M_SEARCH_AUTO_SPEND']=df['M_SP_AB_SPEND']
df['RATE_ON_DIS_CT_ROS']=df['M_ON_DIS_CT_ROS_SPEND']/df['M_ON_DIS_CT_ROS_IMP']
df['RATE_SEARCH_AUTO_CLK']=df['M_SEARCH_AUTO_SPEND']/df['M_SEARCH_AUTO_CLK']
df['RATE_SEARCH_MAN_CLK']=df['M_SEARCH_MAN_SPEND']/df['M_SEARCH_MAN_CLK']
df['RATE_NATIONAL_TV']=df['M_NATIONAL_TV_SPEND']/df['M_NATIONAL_TV_IMP']

## 2). parameters preparation: don't need to change anything at the below cell

In [4]:
# read the threshold, inflection, saturation points for each variable
trans_data = pd.read_excel(open(trans_path, 'rb'), sheet_name='Points')
orig_vars = trans_data['Variable'].apply(lambda x: x.rsplit('_', 5)[0])
inflection = trans_data['Inflection Point']
threshold = trans_data['Threshold Point']
saturation = trans_data['Saturation Point']
inflection_dict = dict(zip(orig_vars, inflection))
threshold_dict = dict(zip(orig_vars, threshold))
saturation_dict = dict(zip(orig_vars, saturation))

# read and pass the parameters required for the simulator
input_file_path = input_file_path
final_model_path = final_model_path
trans_path = trans_path

transformation_method = transformation_method
# the simulation start and end date
simulation_start_end_date = simulation_start_end_date

# the transformation start and end date
modeling_start_end_date = modeling_start_end_date

media_spend_dict = media_spends_dict
granular_media_dict = granular_media_dict
media_cpm_cpi_dict = media_cpm_cpi_dict

apply_adstock = apply_adstock


# Part 2. simulation

## 1). initiatiate the simulator_obj, don't need to change anything at the below cell

In [5]:
# Step 1. call the simulator class to initiate the simulator_obj
simulator_obj = Simulator(df = df,
                  transformation_method = transformation_method,
                  final_model_path = final_model_path,
                  trans_path = trans_path, 
                  simulation_start_end_date = simulation_start_end_date,
                  modeling_start_end_date = modeling_start_end_date,
                  media_spend_dict = media_spend_dict, 
                  granular_media_dict = granular_media_dict, 
                  media_cpm_cpi_dict = media_cpm_cpi_dict,
                  apply_adstock = apply_adstock,
                )

Media       Variable Coeff  New Spend     Historical Spend  \
0       M_ON_DIS_AT_IMP              0.00281    1700000        880,313.45724   
1   M_ON_DIS_CT_ROS_IMP              0.00098     960000        174,151.82333   
2     M_ON_DIS_HPLO_IMP              0.00115      25000        129,757.35484   
3       M_ON_DIS_KW_IMP              0.00611     117000         40,357.54320   
4      M_OFF_DIS_WN_IMP              0.00012    1100000        562,707.96740   
5     M_OFF_DIS_PIN_IMP              0.00450      95000        157,539.04221   
6     M_NATIONAL_TV_IMP             27.92593     900000        365,000.00000   
7           M_SP_AB_CLK              0.00000    1375000              0.00000   
8             M_SBA_CLK              0.00000     940000              0.00000   
9          M_SP_KWB_CLK              0.00000    3200000              0.00000   
10    M_SEARCH_AUTO_CLK              0.11501    1375000        746,418.00000   
11     M_SEARCH_MAN_CLK              0.37317    4140000        303,107.50000   

    Historical Contribution     Historical iRoAS                     Rates  \
0             623,180.92196              0.70791             [11.54769968]   
1                   9.25364              0.00005              [7.72991534]   
2              75,583.41437              0.58250           [1063.58487572]   
3                   0.17232              0.00000             [20.25061064]   
4               9,492.50101              0.01687              [6.99934335]   
5             108,848.25795              0.69093               [6.2809684]   
6             572,896.37829              1.56958          [20151.69660872]   
7                   0.00000              0.00000              [1.49035605]   
8                   0.00000              0.00000              [2.18869612]   
9                   0.00000              0.00000              [1.43975469]   
10             69,745.21656              0.09344              [1.49035605]   
11             68,974.70238              0.22756  [2.18869612, 1.43975469]   

   Rate Unit  Hist on-air days            Threshold           Inflection  \
0        CPM               328        181,742.01869        294,522.55542   
1        CPM               266        189,748.58869        217,512.34714   
2        CPD               122              0.00000         60,897.62972   
3        CPM               215         33,077.98178         40,910.92326   
4        CPM               327        539,113.58564        620,590.35932   
5        CPM               187        132,132.28488        149,281.43727   
6        CPM               365             28.57819             46.31246   
7        CPC                 0              0.00000              0.00000   
8        CPC                 0              0.00000              0.00000   
9        CPC                 0              0.00000              0.00000   
10       CPC               351          1,529.65425          2,205.19732   
11       CPC               343            686.78285            963.49330   

             Saturation  Pct Hist Days Less than Threshold  \
0         407,303.09215                           44.93151   
1         245,276.10558                           98.08219   
2         340,729.53687                           66.57534   
3          48,743.86475                          100.00000   
4         702,067.13300                           97.26027   
5         166,430.58966                           83.83562   
6              64.04674                            1.91781   
7               0.00000                            0.00000   
8               0.00000                            0.00000   
9               0.00000                            0.00000   
10          2,880.74039                           63.56164   
11          1,240.20375                           81.91781   

    Pct Hist Days Btw Threshold and Saturation  \
0                                     45.47945   
1                                      0.82192   
2               

## 2). sample the num_points between threshold and saturation points, you can change the num_points

In [6]:
# Step 2. call the public function to sample the num_points between threshold and saturation points
simulator_obj.sample_daily_metrics(num_points = 5)

start at: 16:56:05
processing Var: M_ON_DIS_AT_IMP ...
completed!

processing Var: M_ON_DIS_CT_ROS_IMP ...
completed!

processing Var: M_ON_DIS_HPLO_IMP ...
completed!

processing Var: M_ON_DIS_KW_IMP ...
completed!

processing Var: M_OFF_DIS_WN_IMP ...
completed!

processing Var: M_OFF_DIS_PIN_IMP ...
completed!

processing Var: M_NATIONAL_TV_IMP ...
completed!

Total processing time: 0:00:02.290987 seconds

Simulation sampling results save to: /Users/vn51u3a/Desktop/ms_new/Modeling/OLS Model/auto folder_structure/models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/simulation_results/Simulation_Sampling_Result.xlsx


## 3). read the simulation result to determine the point_dict
### the point_dict could be the threshold_dict/inflection_dict/saturation_dict or a custom_dict

In [7]:
# Step 3. read the simulation result to determine the specific point dict
# point_dict = threshold_dict
point_dict = inflection_dict
# point_dict = saturation_dict

# The keys in the point_dict should be the subset of the variables used in model, you need to take media_spends_dict into account
# the values are the pacing/daily metrics selected from the 'Simulation_Sampling_Result.xlsx' file above 

custom_dict = {'M_ON_DIS_AT_IMP': 309951.678025852,
                 'M_ON_DIS_CT_ROS_IMP': 209844.4618096357,
                 'M_ON_DIS_HPLO_IMP': 59453.18801410106,
                 'M_ON_DIS_KW_IMP': 41573.10458284371,
                 'M_OFF_DIS_WN_IMP': 663528.21386604,
                 'M_OFF_DIS_PIN_IMP': 147367.6086956522,
                 'M_NATIONAL_TV_IMP': 47.2259093507508,
                 'M_SEARCH_AUTO_CLK': 2321.192714453584,
                 'M_SEARCH_MAN_CLK': 942.2937720329024}
                     
                     
# pass the values for the point_dict, it can be the threshold, inflection, saturation or any values between
point_dict = point_dict

result = simulator_obj.display_and_save_simulation_results(point_dict)

start at: 16:56:08
Simulation results save to: /Users/vn51u3a/Desktop/ms_new/Modeling/OLS Model/auto folder_structure/models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/simulation_results/Simulation_Model_Result.xlsx



Media       Variable Coeff     New Contribution  \
0      M_ON_DIS_AT_IMP              0.00281      2,116,191.76900   
1  M_ON_DIS_CT_ROS_IMP              0.00098      1,139,832.56985   
2    M_ON_DIS_HPLO_IMP              0.00115         48,658.26684   
3      M_ON_DIS_KW_IMP              0.00611        565,506.77037   
4     M_OFF_DIS_WN_IMP              0.00012        329,072.04120   
5    M_OFF_DIS_PIN_IMP              0.00450        115,132.59527   
6    M_NATIONAL_TV_IMP             27.92593      1,008,827.56120   
7    M_SEARCH_AUTO_CLK              0.11501        382,254.05708   
8     M_SEARCH_MAN_CLK              0.37317      1,016,982.26442   
9                Total                  nan      6,722,457.89522   

     Contribution Lift  Historical Contribution   Contribution Ratio  \
0              2.39579            623,180.92196              3.39579   
1        123,175.67481                  9.25364        123,176.67481   
2             -0.35623             75,583.41437              0.64377   
3      3,281,659.41064                  0.17232      3,281,660.41064   
4             33.66653              9,492.50101             34.66653   
5              0.05773            108,848.25795              1.05773   
6              0.76093            572,896.37829              1.76093   
7              4.48072             69,745.21656              5.48072   
8             13.74428             68,974.70238             14.74428   
9                  nan          1,528,730.81847                  nan   

             New Spend  Current Hist Spend Ratio     Historical Spend  \
0      1,700,000.00000                   1.93113        880,313.45724   
1        960,000.00000                   5.51243        174,151.82333   
2         25,000.00000                   0.19267        129,757.35484   
3        117,000.00000                   2.89909         40,357.54320   
4      1,100,000.00000                   1.95483        562,707.96740   
5         95,000.00000                   0.60303        157,539.04221   
6        900,000.00000                   2.46575        365,000.00000   
7      1,375,000.00000                   1.84213        746,418.00000   
8      4,140,000.00000                  13.65852        303,107.50000   
9     10,412,000.00000                       nan      3,359,352.68822   

             New iRoAS  ...     Hist on-air days            Threshold  \
0              1.24482  ...            328.00000        181,742.01869   
1              1.18733  ...            266.00000        189,748.58869   
2              1.94633  ...            122.00000              0.00000   
3              4.83339  ...            215.00000         33,077.98178   
4              0.29916  ...            327.00000        539,113.58564   
5              1.21192  ...            187.00000        132,132.28488   
6              1.12092  ...            365.00000             28.57819   
7              0.27800  ...            351.00000          1,529.65425   
8              0.24565  ...            343.00000            686.78285   
9             12.36751  ...                  nan                  nan   

            Inflection           Saturation  \
0        294,522.55542        407,303.09215   
1        217,512.34714        245,276.10558   
2         60,897.62972        340,729.53687   
3         40,910.92326         48,743.86475   
4        620,590.35932        702,067.13300   
5        149,281.43727        166,430.58966   
6             46.31246             64.04674   
7          2,205.19732          2,880.74039   
8            963.49330          1,240.20375   
9                  nan                  nan   

  Pct Simulation Days Less than Threshold  \
0                                 0.00000   
1                                 0.00000   
2                                93.69863   
3                                61.36986   
4                                30.68493   
5                                72.32877   
6                                 

Populated simulation dataframe save to: /Users/vn51u3a/Desktop/ms_new/Modeling/OLS Model/auto folder_structure/models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/simulation_results/Simulation_df.csv



M_ON_DIS_AT_SPEND      M_ON_DIS_AT_IMP       RATE_ON_DIS_AT  \
index                                                                       
2022-01-01          4,657.53425        403,330.04622              0.01155   
2022-01-02          4,657.53425        403,330.04622              0.01155   
2022-01-03          4,657.53425        403,330.04622              0.01155   
2022-01-04          4,657.53425        403,330.04622              0.01155   
2022-01-05          4,657.53425        403,330.04622              0.01155   
...                         ...                  ...                  ...   
2022-12-27          4,657.53425        403,330.04622              0.01155   
2022-12-28          4,657.53425        403,330.04622              0.01155   
2022-12-29          4,657.53425        403,330.04622              0.01155   
2022-12-30          4,657.53425        403,330.04622              0.01155   
2022-12-31          4,657.53425        403,330.04622              0.01155   

            M_ON_DIS_CT_ROS_SPEND  M_ON_DIS_CT_ROS_IMP   RATE_ON_DIS_CT_ROS  \
index                                                                         
2022-01-01            2,630.13699        340,254.30692              0.00773   
2022-01-02            2,630.13699        340,254.30692              0.00773   
2022-01-03            2,630.13699        340,254.30692              0.00773   
2022-01-04            2,630.13699        340,254.30692              0.00773   
2022-01-05            2,630.13699        340,254.30692              0.00773   
...                           ...                  ...                  ...   
2022-12-27            2,630.13699        340,254.30692              0.00773   
2022-12-28            2,630.13699        340,254.30692              0.00773   
2022-12-29            2,630.13699        340,254.30692              0.00773   
2022-12-30            2,630.13699        340,254.30692              0.00773   
2022-12-31            2,630.13699        340,254.30692              0.00773   

            M_ON_DIS_HPLO_SPEND    M_ON_DIS_HPLO_IMP     RATE_ON_DIS_HPLO  \
index                                                                       
2022-01-01          1,063.58488        102,437.33607          1,063.58488   
2022-01-02          1,063.58488        102,437.33607          1,063.58488   
2022-01-03          1,063.58488        102,437.33607          1,063.58488   
2022-01-04          1,063.58488        102,437.33607          1,063.58488   
2022-01-05          1,063.58488        102,437.33607          1,063.58488   
...                         ...                  ...                  ...   
2022-12-27              0.00000              0.00000          1,063.58488   
2022-12-28              0.00000              0.00000          1,063.58488   
2022-12-29              0.00000              0.00000          1,063.58488   
2022-12-30              0.00000              0.00000          1,063.58488   
2022-12-31              0.00000              0.00000          1,063.58488   

              M_ON_DIS_KW_SPEND  ...       RATE_SP_AB_CLK  \
index                            ...                        
2022-01-01            829.78723  ...              1.49036   
2022-01-02            829.78723  ...              1.49036   
2022-01-03            829.78723  ...              1.49036   
2022-01-04            829.78723  ...              1.49036   
2022-01-05            829.78723  ...              1.49036   
...                         ...  ...                  ...   
2022-12-27              0.00000  ...              1.49036   
2022-12-28              0.00000  ...              1.49036   
2022-12-29              0.00000  ...              1.49036   
2022-12-30              0.00000  ...              1.49036   
2022-12-31              0.00000  ...              1.49036   

             M_SEARCH_MAN_SPEND     M_SEARCH_MAN_CLK  RATE_SEARCH_MAN_CLK  \
index                                                                       
2022-01-01         11,342.46575   

Populated updated stack dataframe save to: /Users/vn51u3a/Desktop/ms_new/Modeling/OLS Model/auto folder_structure/models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/simulation_results/Updated_Stack_df.csv



O_SALE               O_UNIT     O_SALE_TRACEABLE  \
index                                                                       
2019-01-01        361,853.91000         59,807.00000        187,643.69000   
2019-01-02        489,181.39000         80,713.00000        255,425.34000   
2019-01-03        518,177.30000         85,678.00000        269,350.97000   
2019-01-04        560,857.62000         92,795.00000        300,059.85000   
2019-01-05        652,078.00000        107,595.00000        352,024.02000   
...                         ...                  ...                  ...   
2022-12-27              0.00000              0.00000              0.00000   
2022-12-28              0.00000              0.00000              0.00000   
2022-12-29              0.00000              0.00000              0.00000   
2022-12-30              0.00000              0.00000              0.00000   
2022-12-31              0.00000              0.00000              0.00000   

                    O_SALE_CASH     O_UNIT_TRACEABLE          O_UNIT_CASH  \
index                                                                       
2019-01-01        174,210.22000         30,708.00000         29,099.00000   
2019-01-02        233,756.05000         41,714.00000         38,999.00000   
2019-01-03        248,826.33000         44,128.00000         41,550.00000   
2019-01-04        260,797.77000         49,133.00000         43,662.00000   
2019-01-05        300,053.98000         57,576.00000         50,019.00000   
...                         ...                  ...                  ...   
2022-12-27              0.00000              0.00000              0.00000   
2022-12-28              0.00000              0.00000              0.00000   
2022-12-29              0.00000              0.00000              0.00000   
2022-12-30              0.00000              0.00000              0.00000   
2022-12-31              0.00000              0.00000              0.00000   

                   O_SALE_STORE            O_SALE_OG        O_SALE_DOTCOM  \
index                                                                       
2019-01-01        354,199.52000          5,037.41000          2,616.98000   
2019-01-02        476,864.87000          8,213.44000          4,103.08000   
2019-01-03        505,136.25000          9,009.43000          4,031.62000   
2019-01-04        545,985.42000         11,404.14000          3,468.06000   
2019-01-05        636,200.66000         11,527.01000          4,350.33000   
...                         ...                  ...                  ...   
2022-12-27              0.00000              0.00000              0.00000   
2022-12-28              0.00000              0.00000              0.00000   
2022-12-29              0.00000              0.00000              0.00000   
2022-12-30              0.00000              0.00000              0.00000   
2022-12-31              0.00000              0.00000              0.00000   

                   O_UNIT_STORE  ...       INTERCEPT_TERM  \
index                            ...                        
2019-01-01         58,593.00000  ...              1.00000   
2019-01-02         78,788.00000  ...              1.00000   
2019-01-03         83,609.00000  ...              1.00000   
2019-01-04         90,432.00000  ...              1.00000   
2019-01-05        105,074.00000  ...              1.00000   
...                         ...  ...                  ...   
2022-12-27              0.00000  ...              1.00000   
2022-12-28              0.00000  ...              1.00000   
2022-12-29              0.00000  ...              1.00000   
2022-12-30              0.00000  ...              1.00000   
2022-12-31              0.00000  ...              1.00000   

               M_SEARCH_MAN_CLK    M_SEARCH_AUTO_CLK  M_NATIONAL_TV_SPEND  \
index                                                                       
2019-01-01              0.00000              0.00000          1,000.000

Total processing time: 0:00:01.234199 seconds


In [8]:
result[['Media', 
       'New Spend', 'Historical Spend', 'New Contribution', 'Historical Contribution', 'New Daily Metric', 'New Daily Spend',
       'Rates', 'Rate Unit', 'New On-air Days', 'Hist on-air days', 'Threshold', 'Inflection',
       'Saturation', 'Pct Simulation Days Less than Threshold']] 

Media            New Spend     Historical Spend  \
0      M_ON_DIS_AT_IMP      1,700,000.00000        880,313.45724   
1  M_ON_DIS_CT_ROS_IMP        960,000.00000        174,151.82333   
2    M_ON_DIS_HPLO_IMP         25,000.00000        129,757.35484   
3      M_ON_DIS_KW_IMP        117,000.00000         40,357.54320   
4     M_OFF_DIS_WN_IMP      1,100,000.00000        562,707.96740   
5    M_OFF_DIS_PIN_IMP         95,000.00000        157,539.04221   
6    M_NATIONAL_TV_IMP        900,000.00000        365,000.00000   
7    M_SEARCH_AUTO_CLK      1,375,000.00000        746,418.00000   
8     M_SEARCH_MAN_CLK      4,140,000.00000        303,107.50000   
9                Total     10,412,000.00000      3,359,352.68822   

      New Contribution  Historical Contribution     New Daily Metric  \
0      2,116,191.76900            623,180.92196        403,330.04622   
1      1,139,832.56985                  9.25364        340,254.30692   
2         48,658.26684             75,583.41437        102,437.33607   
3        565,506.77037                  0.17232         40,910.92326   
4        329,072.04120              9,492.50101        620,590.35932   
5        115,132.59527            108,848.25795        149,281.43727   
6      1,008,827.56120            572,896.37829            122.35959   
7        382,254.05708             69,745.21656          2,527.66666   
8      1,016,982.26442             68,974.70238          7,265.97341   
9      6,722,457.89522          1,528,730.81847                  nan   

       New Daily Spend                     Rates Rate Unit  \
0          4,657.53425             [11.54769968]       CPM   
1          2,630.13699              [7.72991534]       CPM   
2          1,063.58488           [1063.58487572]       CPD   
3            829.78723             [20.25061064]       CPM   
4          4,347.82609              [6.99934335]       CPM   
5            940.59406               [6.2809684]       CPM   
6          2,465.75342          [20151.69660872]       CPM   
7          3,767.12329              [1.49035605]       CPC   
8         11,342.46575  [2.18869612, 1.43975469]       CPC   
9         32,044.80595                       NaN       NaN   

       New On-air Days     Hist on-air days            Threshold  \
0            365.00000            328.00000        181,742.01869   
1            365.00000            266.00000        189,748.58869   
2             23.00000            122.00000              0.00000   
3            141.00000            215.00000         33,077.98178   
4            253.00000            327.00000        539,113.58564   
5            101.00000            187.00000        132,132.28488   
6            365.00000            365.00000             28.57819   
7            365.00000            351.00000          1,529.65425   
8            365.00000            343.00000            686.78285   
9                  nan                  nan                  nan   

            Inflection           Saturation  \
0        294,522.55542        407,303.09215   
1        217,512.34714        245,276.10558   
2         60,897.62972        340,729.53687   
3         40,910.92326         48,743.86475   
4        620,590.35932        702,067.13300   
5        149,281.43727        166,430.58966   
6             46.31246             64.04674   
7          2,205.19732          2,880.74039   
8            963.49330          1,240.20375   
9                  nan                  nan   

   Pct Simulation Days Less than Threshold  
0                                  0.00000  
1                                  0.00000  
2                                 93.69863  
3                                 61.36986  
4                                 30.68493  
5                                 72.32877  
6                                  0.00000  
7                                  0.00000  
8                                  0.00000  
9                                      nan